# Machine Learning Model for Available Bikes and Bikes Stands

## Contents

* [1. Data Processing](#data_pros) 
.
.
.


In [1]:
#import modules needed
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
# hide ipykernel warnings 
import warnings
warnings.filterwarnings('ignore')
import sqlalchemy as sqla
from sqlalchemy import create_engine

## 1. Data Processing <a class="anchor" id="data_pros"></a>

In [2]:
URI="dublinbikes.cjriubtfwhtu.us-east-1.rds.amazonaws.com"
PORT="3306"
DB="dbbikes"
USER="ellie"
PASSWORD="1994Pamoelda"

In [3]:
engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo = True)

#### Read in the tables

In [4]:
# #read in weather table
# dfweather = pd.read_csv('weather_table.csv', sep=',\s+', delimiter=',', skipinitialspace=True)

In [5]:
# #read in availability table
# dfavailability = pd.read_csv('av_table.csv', sep=',\s+', delimiter=',', skipinitialspace=True)

In [6]:
sql1="""
Select description, dt, temp,temp_min, temp_max, humidity From dbbikes.weather 
"""

In [7]:
sql2="""
Select number, last_update, available_bikes_stands,available_bikes From dbbikes.availability 
"""

In [8]:
dfweather=pd.read_sql_query(sql1, engine)

2021-03-26 09:30:17,383 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-03-26 09:30:17,389 INFO sqlalchemy.engine.base.Engine ()
2021-03-26 09:30:17,521 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-03-26 09:30:17,521 INFO sqlalchemy.engine.base.Engine ()
2021-03-26 09:30:17,756 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-03-26 09:30:17,757 INFO sqlalchemy.engine.base.Engine ()
2021-03-26 09:30:17,995 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2021-03-26 09:30:17,996 INFO sqlalchemy.engine.base.Engine ()
2021-03-26 09:30:18,137 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-03-26 09:30:18,138 INFO sqlalchemy.engine.base.Engine ()
2021-03-26 09:30:18,269 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-03-26 09:30:18,270 INFO sqlalchemy.engine.base.E

In [9]:
dfavailability=pd.read_sql_query(sql2, engine)

2021-03-26 09:30:19,431 INFO sqlalchemy.engine.base.OptionEngine 
Select number, last_update, available_bikes_stands,available_bikes From dbbikes.availability 

2021-03-26 09:30:19,433 INFO sqlalchemy.engine.base.OptionEngine ()


#### Check the number of columns and rows in the tables

In [10]:
dfweather.shape

(2279, 6)

In [11]:
dfavailability.shape

(483687, 4)

#### View the tables

In [12]:
dfweather.head()

,description,dt,temp,temp_min,temp_max,humidity
0,few clouds,2021-03-05 21:23:28,-1,-3,3,86
1,few clouds,2021-03-05 21:39:47,-1,-3,3,86
2,few clouds,2021-03-05 21:57:39,-1,-3,3,86
3,few clouds,2021-03-05 22:10:25,-1,-3,-1,86
4,scattered clouds,2021-03-05 22:27:23,-1,-3,-1,86


In [13]:
dfavailability.head()

,number,last_update,available_bikes_stands,available_bikes
0,2,2021-02-25 20:06:43,9,11
1,2,2021-02-25 20:16:51,9,11
2,2,2021-02-25 20:26:59,9,11
3,2,2021-02-25 20:37:07,9,11
4,2,2021-02-25 20:47:17,9,11


In [14]:
dfavailability.rename(columns = {'number':'StationNumber'}, inplace = True)

#### Check Feature types and change into appropriate types if needed 

In [15]:
dfavailability.dtypes

StationNumber                      int64
last_update               datetime64[ns]
available_bikes_stands             int64
available_bikes                    int64
dtype: object

In [16]:
dfavailability['last_update']=dfavailability['last_update'].apply(pd.to_datetime, format='%Y/%m/%d')

In [17]:
# dfavailability['Day']=dfavailability['last_update'].dt.day_name()

In [18]:
# dfavailability.Day.unique()

In [19]:
dfavailability.nunique()

StationNumber                109
last_update               377640
available_bikes_stands        41
available_bikes               41
dtype: int64

In [20]:
dfavailability.dtypes

StationNumber                      int64
last_update               datetime64[ns]
available_bikes_stands             int64
available_bikes                    int64
dtype: object

In [21]:
dfweather.dtypes

description            object
dt             datetime64[ns]
temp                    int64
temp_min                int64
temp_max                int64
humidity                int64
dtype: object

In [22]:
dfweather['dt']=dfweather['dt'].apply(pd.to_datetime, format='%Y/%m/%d')


In [23]:
# dfweather['Day']=dfweather['dt'].dt.day_name()

In [24]:
# dfweather.Day.unique()

In [25]:
dfweather.nunique()

description      10
dt             2279
temp             17
temp_min         18
temp_max         18
humidity         29
dtype: int64

In [26]:
dfweather['description'] = dfweather['description'].astype('category')

In [27]:
# #do not need icon for model 
# dfweather =dfweather.drop(['icon'], axis=1)

In [28]:
dfweather.dtypes

description          category
dt             datetime64[ns]
temp                    int64
temp_min                int64
temp_max                int64
humidity                int64
dtype: object

#### Check for Missing Data 
-Result: None

In [29]:
dfweather.isnull().sum().sort_values(ascending=False)/len(dfweather)*100

description    0.0
dt             0.0
temp           0.0
temp_min       0.0
temp_max       0.0
humidity       0.0
dtype: float64

In [30]:
dfavailability.isnull().sum().sort_values(ascending=False)/len(dfavailability)*100

StationNumber             0.0
last_update               0.0
available_bikes_stands    0.0
available_bikes           0.0
dtype: float64

#### Check for Duplicate Columns/Rows

-Result: None, will further evaluate below

In [31]:
#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', dfweather.duplicated().sum())

# Check for duplicate rows. 
# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', dfweather[dfweather.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


In [32]:
#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', dfavailability.duplicated().sum())

# Check for duplicate rows. 
# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', dfavailability[dfavailability.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


### Descriptive Stats for Both tables - checking for constant columns

- Results: std >0 - no constant columns

In [33]:
dfavailability.describe().T

,count,mean,std,min,25%,50%,75%,max
StationNumber,483687.0,59.954836,33.773269,2.0,31.0,61.0,90.0,117.0
available_bikes_stands,483687.0,20.815953,9.214039,0.0,14.0,20.0,28.0,40.0
available_bikes,483687.0,11.171036,7.004490,0.0,6.0,10.0,15.0,40.0


In [34]:
dfweather.describe().T

,count,mean,std,min,25%,50%,75%,max
temp,2279.0,7.448442,2.936831,-1.0,5.0,8.0,9.0,15.0
temp_min,2279.0,6.660816,3.053288,-3.0,4.0,7.0,9.0,14.0
temp_max,2279.0,8.222027,2.802361,-1.0,6.0,8.0,10.0,16.0
humidity,2279.0,79.922773,10.735965,43.0,75.0,81.0,87.0,100.0


#### Create new intervals columns which converts the datetime colums into 30 min intervals

In [35]:
dfavailability.head()

,StationNumber,last_update,available_bikes_stands,available_bikes
0,2,2021-02-25 20:06:43,9,11
1,2,2021-02-25 20:16:51,9,11
2,2,2021-02-25 20:26:59,9,11
3,2,2021-02-25 20:37:07,9,11
4,2,2021-02-25 20:47:17,9,11


In [36]:
dfavailability['intervals']=dfavailability['last_update'].dt.round('15min')

In [37]:
dfweather['intervals']=dfweather['dt'].dt.round('15min')

In [38]:
dfweather.head().sort_values('dt')

,description,dt,temp,temp_min,temp_max,humidity,intervals
0,few clouds,2021-03-05 21:23:28,-1,-3,3,86,2021-03-05 21:30:00
1,few clouds,2021-03-05 21:39:47,-1,-3,3,86,2021-03-05 21:45:00
2,few clouds,2021-03-05 21:57:39,-1,-3,3,86,2021-03-05 22:00:00
3,few clouds,2021-03-05 22:10:25,-1,-3,-1,86,2021-03-05 22:15:00
4,scattered clouds,2021-03-05 22:27:23,-1,-3,-1,86,2021-03-05 22:30:00


In [39]:
dfavailability.head().sort_values('last_update')

,StationNumber,last_update,available_bikes_stands,available_bikes,intervals
0,2,2021-02-25 20:06:43,9,11,2021-02-25 20:00:00
1,2,2021-02-25 20:16:51,9,11,2021-02-25 20:15:00
2,2,2021-02-25 20:26:59,9,11,2021-02-25 20:30:00
3,2,2021-02-25 20:37:07,9,11,2021-02-25 20:30:00
4,2,2021-02-25 20:47:17,9,11,2021-02-25 20:45:00


#### Drop Initial Datetime columns as New Intervals Feature will be used to combine the Tables

In [40]:
mergeweather= dfweather.drop(['dt'], axis=1)

In [41]:
mergeavail=dfavailability.drop(['last_update'], axis=1)

#### Sort the Intervals Features before merge

In [42]:
mergeavail.dtypes
mergeweather.dtypes

description          category
temp                    int64
temp_min                int64
temp_max                int64
humidity                int64
intervals      datetime64[ns]
dtype: object

In [43]:
mergeavail.dtypes
mergeweather.dtypes

description          category
temp                    int64
temp_min                int64
temp_max                int64
humidity                int64
intervals      datetime64[ns]
dtype: object

In [44]:
mergeavail=mergeavail.sort_values('intervals')
mergeavail.isnull().sum()

StationNumber             0
available_bikes_stands    0
available_bikes           0
intervals                 0
dtype: int64

In [45]:
mergeweather=mergeweather.sort_values('intervals')
mergeweather.isnull().sum()

description    0
temp           0
temp_min       0
temp_max       0
humidity       0
intervals      0
dtype: int64

In [46]:
BothMerge=pd.merge( mergeavail, mergeweather,how='inner', on='intervals')
# FinalMerge=pd.merge_asof(mergeweather, mergeavail, on='intervals')
# FinalMerge = mergeavail.merge(mergeweather, on='intervals')

In [47]:
BothMerge.dtypes

StationNumber                      int64
available_bikes_stands             int64
available_bikes                    int64
intervals                 datetime64[ns]
description                     category
temp                               int64
temp_min                           int64
temp_max                           int64
humidity                           int64
dtype: object

In [48]:
BothMerge.isnull().sum().sort_values(ascending=False)/len(BothMerge)*100

StationNumber             0.0
available_bikes_stands    0.0
available_bikes           0.0
intervals                 0.0
description               0.0
temp                      0.0
temp_min                  0.0
temp_max                  0.0
humidity                  0.0
dtype: float64

In [49]:
#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', BothMerge.duplicated().sum())

# Check for duplicate rows. 
# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', BothMerge[BothMerge.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  106266
Number of duplicate rows (including first) in the table is: 210713


In [50]:
FinalMerge=BothMerge[~BothMerge.duplicated()]

In [51]:
FinalMerge.shape

(295906, 9)

#### Create a Day Feature from the new merged table so that it is used for the model

In [52]:
FinalMerge['Day']=FinalMerge['intervals'].dt.day_name()

In [53]:
FinalMerge['Hour']=FinalMerge['intervals'].dt.hour

In [54]:
choices=['Morning', 'Afternoon', 'Evening']
conditions=[(FinalMerge['Hour'] > 6) & (FinalMerge['Hour'] < 12), (FinalMerge['Hour'] >= 12) & (FinalMerge['Hour'] < 16),(FinalMerge['Hour'] >= 16) & (FinalMerge['Hour'] < 23)]

In [55]:
FinalMerge['TimeOfDay']= np.select(conditions, choices, default='Night')

In [61]:
FinalMerge.head()

,StationNumber,available_bikes_stands,available_bikes,intervals,description,temp,temp_min,temp_max,humidity,Day,Hour,TimeOfDay
0,107,24,16,2021-03-02 15:30:00,mist,7,6,7,81,Tuesday,15,Afternoon
1,19,19,11,2021-03-02 15:30:00,mist,7,6,7,81,Tuesday,15,Afternoon
2,6,19,1,2021-03-02 15:30:00,mist,7,6,7,81,Tuesday,15,Afternoon
3,72,18,13,2021-03-02 15:30:00,mist,7,6,7,81,Tuesday,15,Afternoon
5,41,12,8,2021-03-02 15:30:00,mist,7,6,7,81,Tuesday,15,Afternoon


In [62]:
FinalMerge.nunique()

StationNumber              109
available_bikes_stands      41
available_bikes             41
intervals                 2260
description                 10
temp                        17
temp_min                    18
temp_max                    18
humidity                    29
Day                          7
Hour                        24
TimeOfDay                    4
dtype: int64

#### 2 models will be created, one for Available Bikes and one for available bike stands - create 2 copies of merged df

In [65]:
availableBikes=FinalMerge.drop(['intervals', 'Hour'], axis=1)
availableStands=FinalMerge.drop(['intervals', 'Hour'], axis=1)

In [66]:
availableBikes.to_csv('availbikes.csv', index=False)
availableStands.to_csv('availstands.csv', index=False)